In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, ConvLSTM2D, Dropout, Flatten, RepeatVector, Reshape, TimeDistributed, Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping


In [36]:
segments = 15
# Set the time step in minutes
time_step = 15
# Set the start and end times for the time window

start_time = '06:00:00'

end_time = '19:00:00'


In [37]:
initial = pd.read_csv("data/bus_running_times_feature_added_all.csv")

In [38]:
busses_new = initial[['trip_id','deviceid','direction','segment','date','start_time','end_time','run_time','run_time_in_seconds']]

In [39]:
busses_new.dropna(inplace=True)

C:\Users\chath\AppData\Local\Temp\ipykernel_14188\50537125.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  busses_new.dropna(inplace=True)


In [40]:
busses_new = busses_new[busses_new['segment'] <= segments]

In [41]:
# Convert the 'date' column to a datetime object
busses_new['date'] = pd.to_datetime(busses_new['date'])

# Create a boolean mask to filter rows with dates on or after 10/1/2022
mask = busses_new['date'] >= '2022-10-01'

# Create the train and test dataframes
busses_train = busses_new[~mask]
busses_test = busses_new[mask]

In [42]:
busses_train.head()

,trip_id,deviceid,direction,segment,date,start_time,end_time,run_time,run_time_in_seconds
0,1.0,262.0,1.0,1.0,2021-10-01,06:39:49,06:40:58,0 days 00:01:09,69.0
1,1.0,262.0,1.0,2.0,2021-10-01,06:42:12,06:45:42,0 days 00:03:30,210.0
2,1.0,262.0,1.0,3.0,2021-10-01,06:45:42,06:53:58,0 days 00:08:16,496.0
3,1.0,262.0,1.0,4.0,2021-10-01,06:54:04,06:57:19,0 days 00:03:15,195.0
4,1.0,262.0,1.0,5.0,2021-10-01,06:57:19,06:58:56,0 days 00:01:37,97.0


In [43]:
def getAvgRuntimeWithSegment(dataframe,time_step,num_segments):
    
    dataframe = dataframe.copy()

    # Convert the 'date' and 'start_time' columns to datetime objects
    dataframe['start_time'] = pd.to_datetime(busses_train['start_time'])
    # Set the start and end times for the time window
    start_time = pd.to_datetime('06:00:00').time()
    end_time = pd.to_datetime('19:00:00').time()

    # Create a list to store the data for the new dataframe
    data = []
    
    # Iterate over the unique dates in the 'date' column
    for date in dataframe['date'].unique():
        # Filter the rows for the current date
        df_date = dataframe[dataframe['date'] == date]
        df_date.loc[:, 'start_time'] = df_date['start_time'].dt.time  # Use .loc to modify the original DataFrame
    
        # Create a time range for the current date with the specified time step
        time_range = pd.date_range(date, date + pd.Timedelta(days=1), freq=f'{time_step}T')
        
        # Iterate over the time range
        for start, end in zip(time_range[:-1], time_range[1:]):
            # Check if the start time is within the specified time window
            if start.time() >= start_time and start.time() < end_time:
                # Filter the rows for the current time window
                mask = (df_date['start_time'] >= start.time()) & (df_date['start_time'] < end.time())
                df_time_window = df_date[mask]
                # Calculate the average run time for each segment in the current time window
                avg_run_time = df_time_window.groupby('segment')['run_time_in_seconds'].mean().reset_index()
    
                # Append the data to the data list
                for row in avg_run_time.itertuples():
                    data.append((date, start.time(), row.segment, row.run_time_in_seconds))
    
    
    # Create a new dataframe from the data list
    df_avg_run_time = pd.DataFrame(data, columns=['date', 'start_time', 'segment', 'avg_run_time'])
    return df_avg_run_time

In [44]:
df_avg_run_time = getAvgRuntimeWithSegment(busses_train,time_step,segments)

C:\Users\chath\AppData\Local\Temp\ipykernel_14188\3838683845.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataframe['start_time'] = pd.to_datetime(busses_train['start_time'])


In [45]:
df_avg_run_time.head(50)

,date,start_time,segment,avg_run_time
0,2021-10-01,06:30:00,1.0,69.0
1,2021-10-01,06:30:00,2.0,210.0
2,2021-10-01,06:45:00,3.0,496.0
3,2021-10-01,06:45:00,4.0,195.0
4,2021-10-01,06:45:00,5.0,97.0
5,2021-10-01,07:00:00,6.0,210.0
6,2021-10-01,07:00:00,7.0,45.0
7,2021-10-01,07:00:00,8.0,99.0
8,2021-10-01,07:00:00,9.0,34.0
9,2021-10-01,07:00:00,10.0,149.0


In [46]:
# To handle outliers in the avg_run_time column of your DataFrame using the absolute deviation around the median method, you can follow these steps:
# 
#     Calculate the median of the avg_run_time column.
#     Calculate the absolute deviations of each value from the median.
#     Calculate the median absolute deviation (MAD) using the absolute deviations.
#     Set a threshold (for example, a multiple of MAD) to identify outliers.
#     Replace the outlier values with a suitable value (e.g., the median).

def handleOutliers(df):
    # Calculate the median of avg_run_time
    median_avg_run_time = df['avg_run_time'].median()
    
    # Calculate the absolute deviations
    df['abs_deviation'] = np.abs(df['avg_run_time'] - median_avg_run_time)
    
    # Calculate the median absolute deviation (MAD)
    mad = df['abs_deviation'].median()
    
    # Set a threshold (for example, 3 times MAD) to identify outliers
    threshold = 3 * mad
    
    # Replace outliers with the median value
    df.loc[df['abs_deviation'] > threshold, 'avg_run_time'] = median_avg_run_time
    
    # Drop the temporary column
    df = df.drop(columns=['abs_deviation'])
    
    return df

In [47]:
df = handleOutliers(df_avg_run_time)

In [48]:
df.head(50)

,date,start_time,segment,avg_run_time
0,2021-10-01,06:30:00,1.0,69.0
1,2021-10-01,06:30:00,2.0,210.0
2,2021-10-01,06:45:00,3.0,158.0
3,2021-10-01,06:45:00,4.0,195.0
4,2021-10-01,06:45:00,5.0,97.0
5,2021-10-01,07:00:00,6.0,210.0
6,2021-10-01,07:00:00,7.0,45.0
7,2021-10-01,07:00:00,8.0,99.0
8,2021-10-01,07:00:00,9.0,34.0
9,2021-10-01,07:00:00,10.0,149.0


In [49]:
def fillTimeSteps(df_avg_run_time, startTime, endTime, time_step, num_segments):
    start_Time = pd.to_datetime(startTime).time()

    end_Time = pd.to_datetime(endTime).time()
    # Create a new DataFrame to store the missing data
    data = []

    # Iterate over the unique dates in the 'date' column
    for date in df_avg_run_time['date'].unique():
        # Filter the rows for the current date
        df_date = df_avg_run_time[df_avg_run_time['date'] == date]

        # Create a time range for the current date with the specified time step
        time_range = pd.date_range(date, date + pd.Timedelta(days=1), freq=f'{time_step}T').time

        # Iterate over the time range
        for t in time_range:
            # Check if the start time is within the specified time window
            if start_Time <= t < end_Time:
                # Check if the current start time is present in the dataframe
                if not (df_date['start_time'] == t).any():
                    # Add rows for the missing start time and segments
                    for segment in range(1, num_segments + 1):
                        data.append((date, t, segment, 0))

    # Create a new DataFrame from the data list
    df_missing = pd.DataFrame(data, columns=['date', 'start_time', 'segment', 'avg_run_time'])

    # Concatenate the new DataFrame with the original DataFrame
    df_avg_run_time = pd.concat([df_avg_run_time, df_missing], ignore_index=True)
    # Sort the rows of the dataframe by the 'date', 'start_time', and 'segment' columns
    df_avg_run_time = df_avg_run_time.sort_values(by=['date', 'start_time', 'segment'])
    return df_avg_run_time

In [50]:
def normalizeSTD(df, num_segments, start_time, end_time, time_step,historical):
    # Filter out zero values when calculating mean for each segment, start time, and weekday
    mean_data = historical.groupby(['segment', 'start_time', df['date'].dt.weekday])['avg_run_time'].mean()
    # Filter out zero values when calculating standard deviation for each segment
    std_data = historical.groupby('segment')['avg_run_time'].std()
    
    # Create a MultiIndex with all combinations of date, start_time, and segment
    idx = pd.MultiIndex.from_product([df['date'].unique(), df['start_time'].unique(), range(1, num_segments + 1)], names=['date', 'start_time', 'segment'])
    
    # Reindex the dataframe using the new MultiIndex
    df = df.set_index(['date', 'start_time', 'segment']).reindex(idx, fill_value=0).reset_index()
    
    df = fillTimeSteps(df, start_time, end_time, time_step, num_segments)
    
    # Normalize values and create a new DataFrame
    normalized_data = []
    for index, row in df.iterrows():
        segment, start_time, weekday = row['segment'], row['start_time'], row['date'].weekday()
        mean_value = mean_data.get((segment, start_time, weekday), 0)  # Default to 0 if not found
        std_value = std_data.get(segment, 1)  # Default to 1 if not found to avoid division by zero
        
        # If the value is non-zero, calculate normalized value; otherwise, keep it as zero
        normalized_value = (row['avg_run_time'] - mean_value) / std_value if row['avg_run_time'] != 0 else 0
        normalized_data.append((row['date'], row['start_time'], segment, normalized_value))
    
    normalized_columns = ['date', 'start_time', 'segment', 'normalized_avg_run_time']
    df_normalized = pd.DataFrame(normalized_data, columns=normalized_columns)
    return df_normalized



In [51]:
df_normalized = normalizeSTD(df, segments, start_time, end_time, time_step,df)

In [52]:
df_normalized.head(150)

,date,start_time,segment,normalized_avg_run_time
0,2021-10-01,06:00:00,1,0.0
1,2021-10-01,06:00:00,2,0.0
2,2021-10-01,06:00:00,3,0.0
3,2021-10-01,06:00:00,4,0.0
4,2021-10-01,06:00:00,5,0.0
...,...,...,...,...
145,2021-10-01,08:15:00,11,0.0
146,2021-10-01,08:15:00,12,0.0
147,2021-10-01,08:15:00,13,0.0
148,2021-10-01,08:15:00,14,0.0


In [53]:
len(df_normalized)

187200

In [54]:
def prepareModelInput(df_avg_run_time, w, s, pred_steps):
    num_samples = len(df_avg_run_time) - (pred_steps - 1)
    # Initialize an empty array to hold the input data
    input_data_x = np.empty((num_samples, w, s, 1))
    input_data_y = np.empty((num_samples, pred_steps, s, 1))
    
    for i in range(len(df_avg_run_time)-((w+pred_steps)*s)):
        # Use array slicing to extract the required data
        sample_data_x = df_avg_run_time.iloc[i:i+w*s, 3].values
        sample_data_y = df_avg_run_time.iloc[i+(w*s):i+(w*s)+pred_steps*s, 3].values
        reshaped_data_x = sample_data_x.reshape(w, s, 1)
        reshaped_data_y = sample_data_y.reshape(pred_steps, s, 1)
        input_data_x[i] = reshaped_data_x
        input_data_y[i] = reshaped_data_y
    return input_data_x, input_data_y

In [55]:
def splitData(input_data_x, input_data_y, test_size):
    # Split the data into training and test sets
    x_train, x_test, y_train, y_test = train_test_split(input_data_x, input_data_y, test_size=test_size, shuffle=False)
    return x_train, x_test, y_train, y_test

In [56]:
def build_model(input_timesteps, output_timesteps, num_links):
    model = Sequential()
    model.add(BatchNormalization(name='batch_norm_0', input_shape=(input_timesteps, num_links, 1, 1)))
    model.add(ConvLSTM2D(name='conv_lstm_1', filters=64, kernel_size=(10, 1), padding='same', return_sequences=True))
    model.add(Dropout(0.21, name='dropout_1'))
    model.add(BatchNormalization(name='batch_norm_1'))
    model.add(ConvLSTM2D(name='conv_lstm_2', filters=64, kernel_size=(5, 1), padding='same', return_sequences=False))
    model.add(Dropout(0.20, name='dropout_2'))
    model.add(BatchNormalization(name='batch_norm_2'))
    model.add(Flatten())
    model.add(RepeatVector(output_timesteps))
    model.add(Reshape((output_timesteps, num_links, 1, 64)))
    model.add(ConvLSTM2D(name='conv_lstm_3', filters=64, kernel_size=(10, 1), padding='same', return_sequences=True))
    model.add(Dropout(0.20, name='dropout_3'))
    model.add(BatchNormalization(name='batch_norm_3'))
    model.add(ConvLSTM2D(name='conv_lstm_4', filters=64, kernel_size=(5, 1), padding='same', return_sequences=True))
    model.add(TimeDistributed(Dense(units=1, name='dense_1', activation='relu')))
    
    optimizer = RMSprop()
    model.compile(loss="mse", optimizer=optimizer)
    
    return model

In [57]:
def trainModel(model, input_data_x, input_data_y, batch_size, epochs, validation_split):
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
    history = model.fit(input_data_x, input_data_y, batch_size=batch_size, epochs=epochs, validation_split=validation_split, callbacks=[early_stopping], verbose=1)
    return history

In [58]:
def evaluateModel(model, x_test, y_test):
    # Make predictions using the test data
    y_pred = model.predict(x_test)
    
    # Calculate the root mean squared error (RMSE)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return rmse